### Date: 5 April 2017

Aim: The aim of this notebook is to compare two cost functions

In [1]:
import sys

In [2]:
sys.path.append("../code/")

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
import sys, traceback
from docopt import docopt
import pandas as pd
import time
import numpy as np
import os
np.random.seed(0)

from common_functions import create_region_df, features_dict, create_feature_combinations, create_df_main
from matrix_factorisation import nmf_features, prepare_df_factorisation,prepare_known_features, create_matrix_factorised, create_prediction

In [5]:
from nnmf_tensorflow import factorize

## Learning the appliance factors for fridge from Austin regin

In [6]:
year=2014
seed = 0
test_home=26
test_home_list = [test_home]
train_regions = ["Austin","Boulder","SanDiego"]
train_fraction_dict = {region:0 for region in train_regions}
train_fraction_dict['Austin']=1.0
test_region = 'Austin'
feature_list = ['energy','home']
appliance="fridge"
feature_comb = ['None']
lat = 2

In [7]:
def f(year, seed, test_home, train_regions, train_fraction_dict,test_region
              , feature_list, appliance, feature_comb, lat, idx_item=None, data_item=None, cost='absolute', use_all=True):
    df, dfc, X_matrix, X_normalised, col_max, col_min, \
    appliance_cols, aggregate_cols, static_features, max_f = prepare_df_factorisation(appliance, year, train_regions,
                                                                                      train_fraction_dict,
                                                                                     test_region, test_home,
                                                                                     feature_list, seed, 3000, 0, use_all)

    idx_user, data_user = prepare_known_features(feature_comb, static_features, X_normalised)

    A = create_matrix_factorised(appliance, test_home, X_normalised)
    X, Y, res = nmf_features(A=A, k=lat, constant=0.01, regularisation=False,
                        idx_user=idx_user, data_user=data_user,
                         idx_item=idx_item, data_item=data_item, MAX_ITERS=15, cost=cost)
    return X, Y, res, df

In [8]:
def f_tf(year, seed, test_home, train_regions, train_fraction_dict,test_region
              , feature_list, appliance, feature_comb, lat, idx_item=None, data_item=None, cost='absolute', use_all=True):
    df, dfc, X_matrix, X_normalised, col_max, col_min, \
    appliance_cols, aggregate_cols, static_features, max_f = prepare_df_factorisation(appliance, year, train_regions,
                                                                                      train_fraction_dict,
                                                                                     test_region, test_home,
                                                                                     feature_list, seed, 3000, 0, use_all)

    idx_user, data_user = prepare_known_features(feature_comb, static_features, X_normalised)

    A = create_matrix_factorised(appliance, test_home, X_normalised)
    A = A.astype('float32')
    X, Y = factorize(A, rank=lat, cost_function=cost) 
    
    return X, Y, None, df

In [9]:
from common_functions import *

In [10]:
preds = {}
errors ={}

In [11]:
import time

In [16]:
for appliance in ['hvac']:
    df, dfc, X_matrix, X_normalised, col_max, col_min, \
    appliance_cols, aggregate_cols, static_features, max_f = prepare_df_factorisation(appliance, year, train_regions,
                                                                                      train_fraction_dict,
                                                                                test_region, test_home_list,
                                                                                     feature_list, seed, 3000, 0, True)
    
    homes = X_matrix.index
    
    if appliance not in preds:
        preds[appliance] = {}
        errors[appliance] = {}

    for cost in ['absolute','relative']:
        if cost not in preds[appliance]:
            preds[appliance][cost] = {}
            errors[appliance][cost] = {}
        for lat in range(2, 3):
            if lat in preds[appliance][cost]:
                continue
            print "*"*20
            o = {}
            for home in homes[:2]:
                print appliance, cost, lat, home
                Y, X, res, df_austin = f(year, seed, [home], train_regions, {"Austin":1.0,"Boulder":0,"SanDiego":0}, 
                          "Austin", feature_list, appliance, feature_comb, lat, cost=cost, use_all = True)
                
                pred_dfcreate_prediction(home, X, Y, X_normalised, appliance, col_max, col_min, appliance_cols):

                pred_df = pd.DataFrame(np.dot(X, Y))
                pred_df.index = X_normalised.index
                o[home]=pred_df

********************
hvac absolute 2 121
hvac absolute 2 135
********************
hvac relative 2 121
hvac relative 2 135


In [17]:
o

{121:             0         1         2         3         4         5         6   \
 121   0.051817  0.136334  0.185234  0.223201  0.136691  0.066049  0.321325   
 135   0.021808  0.066393  0.083954  0.102594  0.060503  0.025226  0.100970   
 222   0.086440  0.230005  0.310714  0.374811  0.228874  0.109445  0.526232   
 364   0.058083  0.170740  0.219552  0.267401  0.159134  0.068925  0.292081   
 370   0.054156  0.138469  0.190922  0.229417  0.141535  0.070177  0.351108   
 410   0.031423  0.100237  0.124010  0.152219  0.088688  0.035045  0.128121   
 545   0.079792  0.245642  0.308985  0.377992  0.222270  0.091524  0.359112   
 661   0.066983  0.203420  0.257528  0.314634  0.185668  0.077627  0.312061   
 871   0.031696  0.080161  0.111155  0.133425  0.082545  0.041323  0.208837   
 936   0.036683  0.121894  0.148013  0.182384  0.105144  0.039521  0.131038   
 946   0.055884  0.185873  0.225603  0.278018  0.160236  0.060156  0.198946   
 1283  0.067229  0.195268  0.252555  0.307243  

In [17]:
            preds[appliance][cost][lat] = pd.DataFrame(o).T
            
            agg = df_austin[aggregate_cols].ix[preds[appliance][cost][lat].index]
            agg.columns = preds[appliance][cost][lat].columns

            gt_appliance = df_austin[appliance_cols].ix[preds[appliance][cost][lat].index]

            gt_fraction = 100*(gt_appliance/agg)
            pred_fraction = 100*(preds[appliance][cost][lat]/agg)
            gt_fraction_dropna = gt_fraction.unstack().dropna()
            pred_fraction_dropna = pred_fraction.unstack().dropna()
            pred_fraction_dropna = pred_fraction_dropna.ix[gt_fraction_dropna.index]

            rms = np.sqrt(mean_squared_error(gt_fraction_dropna, pred_fraction_dropna))
            errors[appliance][cost][lat] = rms
            
            

********************
hvac absolute 2 121
hvac absolute 2 135


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [19]:
X

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.103562,0.263506,0.360711,0.422518,0.246012,0.125303,0.478969,0.696704,0.840013,0.894220,0.638388,0.474660
1,0.134534,0.261492,0.316631,0.396499,0.259507,0.158994,0.550992,0.702104,0.763482,0.877582,0.718754,0.596281


In [20]:
Y

,0,1
0,0.262941,0.326752
1,0.340922,0.318742
2,0.500328,0.474332
3,0.208807,0.360632
4,0.185941,0.479663
5,0.333707,0.000000
6,0.343179,0.403381
7,0.340221,0.321876
8,0.197240,0.189935
9,0.182433,0.156143


In [23]:
pd.DataFrame(np.dot(Y, X))            

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.071190,0.154730,0.198306,0.240654,0.149481,0.084899,0.305978,0.412606,0.470343,0.521879,0.402713,0.319644
1,0.078188,0.173184,0.223898,0.270426,0.166587,0.093397,0.338915,0.461311,0.529733,0.584582,0.446738,0.351882
2,0.115629,0.255874,0.330662,0.399469,0.246179,0.138108,0.500994,0.681610,0.782426,0.863668,0.660331,0.520320
3,0.070142,0.149324,0.189506,0.231215,0.144956,0.083503,0.298717,0.398678,0.450737,0.503204,0.392506,0.314150
4,0.083787,0.174425,0.218947,0.268749,0.170219,0.099563,0.353350,0.466319,0.522407,0.587216,0.463462,0.374273
5,0.034560,0.087934,0.120372,0.140997,0.082096,0.041814,0.159835,0.232495,0.280318,0.298408,0.213035,0.158398
6,0.089809,0.195911,0.251512,0.304939,0.189106,0.107137,0.386632,0.522309,0.596249,0.660878,0.509013,0.403422
7,0.078537,0.173818,0.224638,0.271373,0.167228,0.093807,0.340306,0.463024,0.531537,0.586705,0.448543,0.353418
8,0.045979,0.101640,0.131286,0.158646,0.097813,0.054913,0.199124,0.270772,0.310696,0.343059,0.262432,0.206876
9,0.039900,0.088902,0.115245,0.138991,0.085401,0.047685,0.173413,0.236730,0.272458,0.300163,0.228691,0.179698


In [40]:
pd.DataFrame(errors['fridge'])            

,absolute,relative
2,5.326796,23.600253
3,5.442200,18.708531
4,5.396771,7.529615
5,5.902310,9.146864
6,6.326673,11.312474
7,7.628521,11.829147
8,8.388596,10.174490
9,11.175640,10.679981


In [10]:
from nnmf_tensorflow import factorize

In [11]:
appliance="hvac"
df, dfc, X_matrix, X_normalised, col_max, col_min, \
    appliance_cols, aggregate_cols, static_features, max_f = prepare_df_factorisation(appliance, year, train_regions,
                                                                                      train_fraction_dict,
                                                                                test_region, test_home_list,
                                                                                     feature_list, seed, 3000, 0, True)

In [12]:
appliance

'hvac'

In [13]:
X_normalised = X_normalised.astype('float32')
W, H = factorize(X_normalised, 2)

In [15]:
(pd.DataFrame(np.dot(W, H))*3000).tail()

,0,1,2,3,4,5,6,7,8,9,10,11
84,74.867729,162.652298,206.793289,250.901291,155.426331,89.471634,317.821838,429.687653,487.736877,539.394287,415.734070,333.283051
85,184.830048,392.809723,486.433014,593.548889,369.545044,221.629608,760.041809,1021.026672,1138.304932,1259.015625,980.133972,808.925354
86,97.522346,217.838852,285.820862,344.488800,212.132050,116.035759,430.783203,586.870056,680.264160,752.210022,573.095947,443.605408
87,143.909790,304.383209,374.712952,457.817139,285.362244,172.686859,587.663879,788.330078,875.292358,968.138489,755.413513,627.515808
88,217.139923,471.087372,597.958923,725.752869,449.722931,259.551819,919.939697,1243.246826,1409.655640,1558.966919,1202.293701,965.585388


In [16]:
W, H = factorize(X_normalised, 2, cost_function='relative')

In [17]:
(pd.DataFrame(np.dot(W, H))*3000).tail()

,0,1,2,3,4,5,6,7,8,9,10,11
84,299.194458,266.451508,414.750061,478.568146,309.457306,252.858841,586.364746,733.760315,984.917603,1119.563110,831.785583,613.349915
85,330.291931,282.776764,460.767090,523.415344,337.400024,278.559906,671.394714,825.619690,1127.792847,1324.390869,951.595276,700.113525
86,200.837860,175.373108,279.297821,319.744171,206.432663,169.556580,400.988586,497.326538,673.556458,778.642395,568.572815,418.774323
87,251.158966,215.551483,350.240204,398.238403,256.758545,211.847885,509.429138,627.095276,855.724609,1003.011414,722.070740,531.316711
88,396.947937,351.557129,550.757080,634.087646,409.839844,335.373901,782.074219,976.170898,1313.659668,1500.522705,1109.269653,817.692322


In [ ]:
from plot_functions import *

In [ ]:
for appliance in ['hvac','fridge']:
    latexify()
    pd.DataFrame(errors[appliance]).plot(kind='bar',rot=0, colors=tableau20blind[::2])
    plt.xlabel("\# latent factors")
    plt.title(appliance)
    plt.ylabel("Train RMS Error")
    ax = plt.gca()
    rects = ax.patches
    autolabel(rects, ax)
    format_axes(ax)
    plt.savefig("../results/cost_%s.png" %appliance, dpi=300, bbox_inches="tight")

In [ ]:
gt = X_normalised*3000

In [ ]:
gt.head()

In [ ]:
pred.head()

In [ ]:
X_austin ={}
Y_austin= {}
for appliance in ['fridge','hvac']:
    X_austin[appliance], Y_austin[appliance], res_sd, df_austin = f(year, seed, [22], train_regions, {"Austin":1.0,"Boulder":0,"SanDiego":0}, 
              "Austin", feature_list, appliance, feature_comb, lat)

In [ ]:
from common_functions import valid_homes_data

### Oracle

In [ ]:
year=2014
seed = 0
train_regions = ["Austin","Boulder","SanDiego"]
train_fraction_dict = {region:0 for region in train_regions}
train_fraction_dict['SanDiego']=1.0
test_region = 'SanDiego'
feature_list = ['energy','home']
feature_comb = ['None']
lat = 2

In [ ]:
import sys

In [ ]:
def compute_oracle(lat):
    out_oracle = {}
    for appliance in ['fridge','hvac']:

        out_oracle[appliance] = {}


        for i, test_home in enumerate(valid_homes_data['SanDiego'][appliance][:]):
            #print test_home
            test_home_list=[test_home]
            df, dfc, X_matrix, X_normalised, col_max, col_min, appliance_cols, aggregate_cols, static_features, max_f = prepare_df_factorisation(appliance, year, train_regions,
                                                                                              train_fraction_dict,
                                                                                             test_region, test_home_list,
                                                                                             feature_list, seed, 3000, 0)

            idx_user, data_user = prepare_known_features(feature_comb, static_features, X_normalised)
            A = create_matrix_factorised(appliance, test_home_list, X_normalised)



            data_items = None
            idx_items = None
            X, Y, res = nmf_features(A=A, k=lat, constant=0.01, regularisation=False,
                                idx_user=idx_user, data_user=data_user,
                                 idx_item=idx_items, data_item=data_items, MAX_ITERS=10)
            pred_df = create_prediction(test_home, X, Y, X_normalised, appliance,
                                   col_max, col_min, appliance_cols)
            out_oracle[appliance][test_home]=pred_df
    return out_oracle

### Baseline (k-fold on same region, no transfer)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
def compute_baselines(lat):
    out_baseline = {}
    for appliance in ['fridge','hvac']:
    #for appliance in ['fridge']:

        out_baseline[appliance] = {}
        for reuse_factors in [True, False]:
        #for reuse_factors in [True]:
            out_baseline[appliance][reuse_factors] = {}
            n_splits=3
            kf = KFold(n_splits=n_splits)
            all_homes = valid_homes_data['SanDiego'][appliance][:]
            temp = {}
            for fold, (test_home_idx, train_home_idx) in enumerate(kf.split(all_homes)):
                temp[fold] = {}

                test_home_list = all_homes[test_home_idx]
                train_home_list = all_homes[train_home_idx]
                #print appliance, reuse_factors, test_home_list, fold
                df, dfc, X_matrix, X_normalised, col_max, col_min, appliance_cols, aggregate_cols, static_features, max_f = prepare_df_factorisation(appliance, year, train_regions,
                                                                                                  train_fraction_dict,
                                                                                                 test_region, test_home_list,
                                                                                                 feature_list, seed, 3000, 0)

                idx_user, data_user = prepare_known_features(feature_comb, static_features, X_normalised)
                A = create_matrix_factorised(appliance, test_home_list, X_normalised)

                if reuse_factors:
                    from collections import OrderedDict
                    data_items = OrderedDict()
                    idx_items = OrderedDict()

                    for count in range(X_austin[appliance].shape[0]):
                        data_items['lat_%d' %count] = np.array(X_austin[appliance][count]).flatten()
                        idx_items['lat_%d' %count] = np.array(range(X_austin[appliance][count].size))
                else:

                    data_items = None
                    idx_items = None




                X, Y, res = nmf_features(A=A, k=lat, constant=0.01, regularisation=False,
                                    idx_user=idx_user, data_user=data_user,
                                     idx_item=idx_items, data_item=data_items, MAX_ITERS=10)
                for test_home in test_home_list:
                    pred_df = create_prediction(test_home, X, Y, X_normalised, appliance,
                                           col_max, col_min, appliance_cols)
                    temp[fold][test_home]=pred_df
            for home in all_homes:
                p =[]

                for fold in range(n_splits):
                    if home in pd.DataFrame(temp[fold]).T.index:
                        p.append(pd.DataFrame(temp[fold]).T.ix[home])
                out_baseline[appliance][reuse_factors][home]=pd.DataFrame(p).mean()
    return out_baseline

In [ ]:
ob = {}
ora ={}

In [ ]:
for k in [2,3, 4]:
    print k, "baseline"
    ob[k] = compute_baselines(k)
    print k, "oracle"
    ora[k] = compute_oracle(k)

In [ ]:
pd.DataFrame(out_baseline['fridge'][True]).T

In [ ]:
from common_functions import compute_rmse_fraction

In [ ]:
res = OrderedDict()
for lat in range(2, 5):
    res[lat]=OrderedDict()
    for appliance in ['hvac','fridge']:
        res[lat][appliance] = OrderedDict()
        res[lat][appliance]['Oracle'] = compute_rmse_fraction(appliance, pd.DataFrame(ora[lat][appliance]).T,
                                                                 "SanDiego")[2]
        for reuse_factors in [True, False]:
            if reuse_factors:
                string = "Transferred from Austin"
            else:
                string = "Learnt from SD"
            res[lat][appliance][string] =compute_rmse_fraction(appliance,pd.DataFrame(ob[lat][appliance][reuse_factors]).T,
                                                             "SanDiego")[2]
    



In [ ]:
%matplotlib inline

In [ ]:
pd.Panel(res)

In [ ]:
pd.DataFrame(res[4])

In [ ]:
pd.DataFrame(res[3])

In [ ]:
pd.DataFrame(res[2])

In [ ]:
from plot_functions import autolabel, latexify, tableau20blind, format_axes
latexify()
r={}
for appliance in ['hvac','fridge']:
    r[appliance]={}
    for l in range(2, 5):
        r[appliance][l]=pd.DataFrame(res[l][appliance]).squeeze()
        
       
#plt.savefig("/Users/nipunbatra/git/scalable-nilm/results/transfer.png",bbox_inches="tight")
#plt.savefig("/Users/nipunbatra/git/scalable-nilm/results/transfer.pdf",bbox_inches="tight")

In [ ]:
for appliance in ['fridge','hvac']:
    ax = pd.DataFrame(r[appliance]).T.plot()
    format_axes(ax)
    ax.set_title(appliance)
    ax.set_xlabel("\#latent factors")
    ax.set_ylabel("RMS error")
    plt.savefig("/Users/nipunbatra/git/scalable-nilm/results/transfer_latent_%s.png" %appliance,bbox_inches="tight")

In [ ]:
from plot_functions import autolabel, latexify, tableau20blind, format_axes
latexify()
ax = pd.DataFrame(res).T[['Oracle','Learnt from SD', 'Transferred from Austin']].plot(kind="line",rot=0, colors=tableau20blind[::2])
plt.ylabel("RMS~error \n(Lower is better)")
plt.legend(loc=3)
rects = ax.patches
autolabel(rects, ax)
format_axes(ax)
#plt.savefig("/Users/nipunbatra/git/scalable-nilm/results/transfer.png",bbox_inches="tight")
#plt.savefig("/Users/nipunbatra/git/scalable-nilm/results/transfer.pdf",bbox_inches="tight")

In [ ]:
difference_fraction = {}
for appliance in ['hvac','fridge']:
    difference_fraction[appliance] = {}
    for reuse_factors in [True, False]:
        if reuse_factors:
            string = "Transferred from Austin"
        else:
            string = "Learnt from SD"
        gtf, prf, rms = compute_rmse_fraction(appliance,pd.DataFrame(out[appliance][reuse_factors]).T,
                                                             "SanDiego")
        errors[appliance][string] = (prf-gtf).mean().mean()


In [ ]:
appliance="hvac"
gtf, prf, rms = compute_rmse_fraction(appliance,pd.DataFrame(out[appliance][False]).T,
                                                             "SanDiego")

In [ ]:
gtf_au, prf_au, rms = compute_rmse_fraction(appliance,pd.DataFrame(out[appliance][True]).T,
                                                             "SanDiego")

In [ ]:
prf.describe()

In [ ]:
prf_au.describe()

In [ ]:
X_sd ={}
Y_sd= {}
for appliance in ['fridge','hvac']:
    X_sd[appliance], Y_sd[appliance], res_sd, df_austin = f(year, seed, 54, train_regions, {"Austin":0.0,"Boulder":0,"SanDiego":1.0}, 
              "SanDiego", feature_list, appliance, feature_comb, lat)

In [ ]:
pd.DataFrame(X_sd['fridge']).T.describe()

In [ ]:
pd.DataFrame(X_austin['fridge']).T.describe()

In [ ]:
X_fridge = pd.DataFrame({"Austin":np.array(X_fridge_austin).flatten(),'SanDiego':np.array(X_fridge_sd).flatten()})
X_hvac = pd.DataFrame({"Austin":np.array(X_hvac_austin).flatten(),'SanDiego':np.array(X_hvac_sd).flatten()})

In [ ]:
X_fridge[:12].plot()
plt.xlabel("Month of year")
plt.ylabel("Latent factors for fridge")

In [ ]:
X_hvac[6:].plot()
plt.xlabel("Month of year")
plt.ylabel("Latent factors for HVAC")

In [ ]:
X_fridge.plot()

### Using the fridge latent factors from Austin region

In [ ]:
pd.DataFrame(X).head()

In [ ]:
pd.DataFrame(X_fridge_austin).head()

##### X_fridge_sd, Y_fridge_sd, res_sd, df_fridge_sd = f(year, seed, 54, train_regions, {"Austin":0,"Boulder":0,"SanDiego":1}, 
              "SanDiego", feature_list, "fridge", feature_comb, lat)
X_hvac_sd, Y_hvac_sd, res_sd, df_hvac_sd = f(year, seed, 54, train_regions, {"Austin":0,"Boulder":0,"SanDiego":1}, 
              "SanDiego", feature_list, "hvac", feature_comb, lat)